##### This notebook is dedicated to preparing the data and training different regression models to forecast weekly sales.

### Load data
Load the cleaned dataset created in the previous notebook (`train_merged_clean.csv`).

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error 
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

df = pd.read_csv("../data/processed/train_merged_clean.csv")

### Check Features
- Inspect numeric and categorical columns
- Check for missing values

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420285 entries, 0 to 420284
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         420285 non-null  int64  
 1   Dept          420285 non-null  int64  
 2   Date          420285 non-null  object 
 3   Year          420285 non-null  int64  
 4   Month         420285 non-null  int64  
 5   Quarter       420285 non-null  int64  
 6   ISO_Week      420285 non-null  int64  
 7   Weekly_Sales  420285 non-null  float64
 8   IsHoliday     420285 non-null  bool   
 9   Type          420285 non-null  object 
 10  Size          420285 non-null  int64  
 11  Temperature   420285 non-null  float64
 12  Fuel_Price    420285 non-null  float64
 13  CPI           420285 non-null  float64
 14  Unemployment  420285 non-null  float64
dtypes: bool(1), float64(5), int64(7), object(2)
memory usage: 45.3+ MB


In [3]:
df.isnull().sum()

Store           0
Dept            0
Date            0
Year            0
Month           0
Quarter         0
ISO_Week        0
Weekly_Sales    0
IsHoliday       0
Type            0
Size            0
Temperature     0
Fuel_Price      0
CPI             0
Unemployment    0
dtype: int64

In [4]:
df.describe()

,Store,Dept,Year,Month,Quarter,ISO_Week,Weekly_Sales,Size,Temperature,Fuel_Price,CPI,Unemployment
count,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000,420285.000000
mean,22.195477,44.242771,2010.968443,6.449709,2.482920,25.827729,16030.329773,136749.569176,60.090474,3.360888,171.212152,7.960077
std,12.787213,30.507197,0.796893,3.243394,1.071464,14.152442,22728.500149,60992.688568,18.448260,0.458523,39.162280,1.863873
min,1.000000,1.000000,2010.000000,1.000000,1.000000,1.000000,0.000000,34875.000000,-2.060000,2.472000,126.064000,3.879000
25%,11.000000,18.000000,2010.000000,4.000000,2.000000,14.000000,2117.560000,93638.000000,46.680000,2.933000,132.022667,6.891000
50%,22.000000,37.000000,2011.000000,6.000000,2.000000,26.000000,7659.090000,140167.000000,62.090000,3.452000,182.350989,7.866000
75%,33.000000,74.000000,2012.000000,9.000000,3.000000,38.000000,20268.380000,202505.000000,74.280000,3.738000,212.445487,8.567000
max,45.000000,99.000000,2012.000000,12.000000,4.000000,52.000000,693099.360000,219622.000000,100.140000,4.468000,227.232807,14.313000


### One-Hot Encoding and Drop Irrelevant Column
- Transform categorical variable `Type` using one-hot encoding.  
- `drop_first=True` avoids collinearity for linear models.
- Drop `Date` because I already have `Year`, `Month`, `Quarter`, `ISO_Week`.

In [5]:
print(df["Type"].unique())

['A' 'B' 'C']


In [6]:
df_dummies = pd.get_dummies(df, columns=["Type"], drop_first=True)

In [7]:
print(df_dummies.head())

   Store  Dept        Date  Year  Month  Quarter  ISO_Week  Weekly_Sales  \
0      1     1  2010-02-05  2010      2        1         5      24924.50   
1      1     1  2010-02-12  2010      2        1         6      46039.49   
2      1     1  2010-02-19  2010      2        1         7      41595.55   
3      1     1  2010-02-26  2010      2        1         8      19403.54   
4      1     1  2010-03-05  2010      3        1         9      21827.90   

   IsHoliday    Size  Temperature  Fuel_Price         CPI  Unemployment  \
0      False  151315        42.31       2.572  211.096358         8.106   
1       True  151315        38.51       2.548  211.242170         8.106   
2      False  151315        39.93       2.514  211.289143         8.106   
3      False  151315        46.63       2.561  211.319643         8.106   
4      False  151315        46.50       2.625  211.350143         8.106   

   Type_B  Type_C  
0   False   False  
1   False   False  
2   False   False  
3   False   

In [8]:
df = df_dummies.drop(columns=["Date"])

### Split Train / Validation / Test

- I create a small **final test set** (10%) to evaluate the model after tuning.
- I use the remaining data to create **training (80%)** and **validation (20%)** sets.

In [9]:
X = df.drop(columns=['Weekly_Sales'])
y = df['Weekly_Sales']
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=42)
print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)
print("Test shape:", X_test.shape)

Train shape: (285793, 14)
Validation shape: (71449, 14)
Test shape: (63043, 14)


### Scale the features and Train Multiple Models

Linear models and SGD need scaling, I use `StandardScaler`
- Linear models: LinearRegression, Ridge, Lasso, ElasticNet.
- SGDRegressor, more suitable for large datasets.
- Ensemble methods: RandomForestRegressor, GradientBoostingRegressor.
- I compare **RMSE on validation set**.

In [10]:
linear_models = {
    "LinearRegression": Pipeline([("scaler", StandardScaler()), ("model", LinearRegression())]),
    "Ridge": Pipeline([("scaler", StandardScaler()), ("model", Ridge(max_iter=10000))]),
    "Lasso": Pipeline([("scaler", StandardScaler()), ("model", Lasso(max_iter=10000))]),
    "ElasticNet": Pipeline([("scaler", StandardScaler()), ("model", ElasticNet(max_iter=10000))]),
    "SGDRegressor": Pipeline([("scaler", StandardScaler()), ("model", SGDRegressor(max_iter=5000, tol=1e-4, random_state=42))])
}
tree_models = {
    "RandomForest": RandomForestRegressor(n_estimators=300, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, random_state=42)
}
models = {**linear_models, **tree_models}    
    
rmse_results={}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    rmse_results[name] = rmse
    print(f"{name} RMSE on validation set: {rmse:.2f}")

LinearRegression RMSE on validation set: 21679.08
Ridge RMSE on validation set: 21679.08
Lasso RMSE on validation set: 21679.02
ElasticNet RMSE on validation set: 21817.93
SGDRegressor RMSE on validation set: 21698.71
RandomForest RMSE on validation set: 3771.29
GradientBoosting RMSE on validation set: 10681.57


### Key Observations

- **Linear Models** (LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor)  
  - All give similar RMSE (~21,600).  
  - Linear relationships only; regularization has little effect with default parameters.  

- **Tree-Based Models** (RandomForest, GradientBoosting)  
  - Perform much better (RandomForest ~3,771, GradientBoosting ~9,688).  
  - Capture non-linear patterns and feature interactions naturally.  

- **Reason for Gap**  
  - The target likely depends on non-linear relationships.  
  - Linear models cannot model these without feature engineering.  
  - Tree ensembles handle complexity directly, leading to lower RMSE.

### Hyperparameters tuning

In this step, I focus on improving **Random Forest** and **Gradient Boosting** models by tuning their hyperparameters. 

I use `HalvingRandomSearchCV` with cross-validation to efficiently explore different combinations of parameters:

- **Random Forest:** `n_estimators`, `max_depth`, `min_samples_split`, `min_samples_leaf`.
- **Gradient Boosting:** `n_estimators`, `learning_rate`, `max_depth`, `min_samples_split`, `min_samples_leaf`.

HalvingRandomSearchCV speeds up tuning by discarding poor hyperparameter candidates early and allocating more resources only to the promising ones. It is more efficient than `RandomizedSearchCV`, especially on big data and complex models.

These two tree-based models are selected because, in our earlier evaluation, they significantly outperformed linear models in terms of RMSE. Focusing on them allows me to maximize predictive performance while keeping the process efficient.

In [11]:
rf = RandomForestRegressor(random_state=42)

rf_param_grid = {
    "n_estimators": [100, 200, 300, 400, 500],
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

rf_search = HalvingRandomSearchCV(
    estimator=rf,
    param_distributions=rf_param_grid,
    factor=3,  
    scoring="neg_root_mean_squared_error",
    cv=5, 
    resource='n_samples',
    min_resources=20000,
    max_resources='auto',
    n_jobs=-1,
    verbose=2
)

start_time = time.time()
rf_search.fit(X_train, y_train)
end_time = time.time()

print(f"Total elapsed time: {end_time - start_time:.2f} seconds")

rf_best = rf_search.best_estimator_
rf_preds = rf_best.predict(X_val)
print("Random Forest Best Params:", rf_search.best_params_)
print("Random Forest Validation RMSE:", np.sqrt(mean_squared_error(y_val, rf_preds)))

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 20000
max_resources_: 285793
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 14
n_resources: 20000
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 1
n_candidates: 5
n_resources: 60000
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 2
n_candidates: 2
n_resources: 180000
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Total elapsed time: 4064.75 seconds
Random Forest Best Params: {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_depth': None}
Random Forest Validation RMSE: 3831.9476672197047


In [12]:
gb = GradientBoostingRegressor(random_state=42)

gb_param_grid = {  
    "n_estimators": [100, 200, 300, 400, 500],
    "learning_rate": [0.01, 0.05, 0.1, 0.2],
    "max_depth": [3, 5, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}

gb_search = HalvingRandomSearchCV(
    estimator=gb,
    param_distributions=gb_param_grid,
    factor=3,
    scoring="neg_root_mean_squared_error",
    cv=5, 
    resource='n_samples',
    min_resources=20000,
    max_resources='auto',
    n_jobs=-1,
    random_state=42,
    verbose=2
)

start_time = time.time()
gb_search.fit(X_train, y_train)
end_time = time.time()

print(f"Total elapsed time: {end_time - start_time:.2f} seconds")

gb_best = gb_search.best_estimator_
gb_preds = gb_best.predict(X_val)
print("Gradient Boosting Best Params:", gb_search.best_params_)
print("Gradient Boosting Validation RMSE:", np.sqrt(mean_squared_error(y_val, gb_preds)))

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 20000
max_resources_: 285793
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 14
n_resources: 20000
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 1
n_candidates: 5
n_resources: 60000
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 2
n_candidates: 2
n_resources: 180000
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Total elapsed time: 2245.30 seconds
Gradient Boosting Best Params: {'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 10, 'learning_rate': 0.1}
Gradient Boosting Validation RMSE: 3005.6031338390785


Since `HalvingRandomSearchCV` uses successive elimination, not all parameter combinations are fully tested.
The best `n_estimators=400` does not guarantee that `500` would perform worse.
I plan to further improve validation performance by testing higher values of the `n_estimators` hyperparameter.

In [13]:
gb_best_params = {'n_estimators': 400, 'min_samples_split': 5, 
                  'min_samples_leaf': 2, 'max_depth': 10, 'learning_rate': 0.1}

for n in [400, 450, 500]:
    params = gb_best_params.copy()
    params['n_estimators'] = n

    gb = GradientBoostingRegressor(**params, random_state=42)
    gb.fit(X_train, y_train)
    
    preds = gb.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, preds))
    print(f"n_estimators={n} -> RMSE: {rmse:.2f}")

n_estimators=400 -> RMSE: 3005.60
n_estimators=450 -> RMSE: 2966.98
n_estimators=500 -> RMSE: 2934.58


### Final Model Selection  

After hyperparameter tuning and additional testing with different values of `n_estimators`,  
the **GradientBoostingRegressor** has been selected as the final model.  

The chosen best parameters are:  
- `n_estimators = 500`  
- `min_samples_split = 5`  
- `min_samples_leaf = 2`  
- `max_depth = 10`  
- `learning_rate = 0.1`  

This configuration provided the best validation performance and will be used for the final test evaluation.

### Test Set evaluation

In [14]:
best_params = {
    'n_estimators': 500,
    'min_samples_split': 5,
    'min_samples_leaf': 2,
    'max_depth': 10,
    'learning_rate': 0.1
}

final_model = GradientBoostingRegressor(**best_params, random_state=42)
final_model.fit(X_train, y_train)

final_preds = final_model.predict(X_test)
final_rmse = np.sqrt(mean_squared_error(y_test, final_preds))

model_name = "GradientBoostingRegressor"
print(f"{model_name} RMSE on test set: {final_rmse:.2f}")

GradientBoostingRegressor RMSE on test set: 2795.82


### Save Final Model

In [15]:
import joblib

joblib.dump(final_model, "final_model_gb.pkl")
print("Final GradientBoostingRegressor saved to final_model_gb.pkl")

Final GradientBoostingRegressor saved to final_model_gb.pkl


In [16]:
# To verify:
loaded_model = joblib.load("final_model_gb.pkl")

loaded_preds = loaded_model.predict(X_test)
loaded_rmse = np.sqrt(mean_squared_error(y_test, loaded_preds))
print(f"Reloaded model RMSE on test set: {loaded_rmse:.2f}")

Reloaded model RMSE on test set: 2795.82


#### Forecast results

In [17]:
forecast_df = pd.DataFrame({
    "t": range(len(final_preds)),   # period index (0,1,2,...)
    "demand": final_preds           # forecasted sales
})

# Save to processed folder for optimization step
forecast_path = "../data/processed/forecasts_single_sku.csv"
forecast_df.to_csv(forecast_path, index=False)
print("Forecasts saved to", forecast_path)

Forecasts saved to ../data/processed/forecasts_single_sku.csv
